In [3]:
%load_ext autoreload
%autoreload 2
# the imported files are updated each time a cell is executed (good if functions in py files are added and modified)

import numpy as np
import pandas as pd
import plotly.graph_objects as go
import sys
import os

from scipy.spatial.transform import Rotation as R

sys.path.insert(0, '..')
sys.path.insert(0, '...')

from Python.LMC_util import make_LMC, check_LMC_Hand_visibility, make_hand_poses
from Python.LMC_util import fingers_idx_LMC

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


# Load and process data

In [4]:
# General Settings
# Data Path
current_dir = os.getcwd()
data_path = 'data/LMC_raw'
save_path = 'data/LMC_rayTracing'

# LMC Settings
# Input parameter
finger_radius = 10 # mm

# LMC settings
num_LMC = 1
LMC_H = 400 # mm
LMC_alpha1 = (180-150) / 2 * np.pi / 180
LMC_alpha2 = (180-120) / 2 * np.pi / 180
LMC_position = np.array([0,0,0]) # in mm
LMC_orientation = np.array([0,0,0]) # in deg
LMC = make_LMC(LMC_loc=LMC_position, LMC_orient=LMC_orientation, LMC_H=LMC_H, LMC_alpha1=LMC_alpha1, LMC_alpha2=LMC_alpha2) 

# PROCES DATA
for subject in os.listdir(os.path.join(os.getcwd(),  'data', 'LMC_raw')):
    if os.path.isdir(os.path.join(os.getcwd(),  'data', 'LMC_raw', subject)):
        print('_________________________________________________________________________')
        print(subject)
        for trialtype in ['Naiv', 'Opti']:
            print('    ', trialtype)
            data_path = os.path.join(os.getcwd(),  'data', 'LMC_raw', subject, trialtype)
            save_path = os.path.join(os.getcwd(),  'data', 'LMC_rayTracing', subject, trialtype)
            # load files
            for fi, file in enumerate(os.listdir(os.path.join(current_dir, data_path))):
                if '_Trial' in file or '_Static' in file:
                    print('        ', file)
                    # load trial
                    filename = os.path.join(current_dir, data_path, file)
                    df = pd.read_csv(filename,index_col=False,sep=r',|\t', engine='python')
                    df = df.replace('\t','', regex=True)

                    # to numpy
                    posDatas_ = df.to_numpy()
                    posDatas = posDatas_[:, [4, 5, 6] + [13+i for i in range(94-13)]] # cut away unnecessary data
                    posDatas = posDatas.reshape(posDatas.shape[0], int(posDatas.shape[1]/3), 3)
                    posDatas = posDatas[:, [i for i in range(posDatas.shape[1]) if i not in [0, 2, 3, 8, 13, 18]]] # cut away unnecessary markers

                    occlusions = np.zeros([posDatas.shape[0], 5])
                    for f in range(posDatas.shape[0]):  #for each frame
                        # rotate data such that z points up
                        frame = (R.from_euler('XYZ', [90, 0, 0], degrees=True).as_matrix().squeeze() @ posDatas[f].T).T
                        frame = frame.astype(np.float64) # otherwise there is a type error for some files
                        # transform data in global frame not necessary as we only have one LMC
                        if pd.isnull(frame).any(): # np.isnan(frame).any gives an error for the 'PRAM' subject the LMC did not detect the hand:
                            occlusions[f, :] = np.array([-1]*5)
                            continue
                        
                        # get data for ray tracing
                        handMarkers, palmMarkers, palmPlnNormals, palmCentroids, fingers, forearm_vecs = make_hand_poses(np.array([frame]))   
                        occlusions_, _ = check_LMC_Hand_visibility(LMC, pos_markers=handMarkers[0], fingers_idx=fingers_idx_LMC, finger_lines=fingers[0], 
                                                                    finger_radius=finger_radius, palm_plane_normal=palmPlnNormals[0], 
                                                                    palm_centroid=palmCentroids[0], palm_markers=palmMarkers[0], forearm_vec=forearm_vecs[0],
                                                                    fig = None, verbose=0)
                        occlusions[f, :] = occlusions_[2:]
                    
                    fingers = ['thumb', 'index', 'middle', 'ring', 'pinky']
                    for finger in range(5):
                        df['visibility_'+fingers[finger]] = occlusions[:, finger]
                    savefilename = os.path.join(current_dir, save_path, file)
                    df.to_csv(savefilename, header=True, index=None, sep=' ', mode='w', quoting=1, quotechar=' ')
                    savefilename = os.path.join(current_dir, save_path, file[:-4] + '.pkl')
                    df.to_pickle(savefilename) #long therm storage

_________________________________________________________________________
Subject1
     Naiv
         Interp_LMC_Naiv_Static101.txt
         Interp_LMC_Naiv_Static102.txt
     Opti
         Interp_LMC_Opti_Static101.txt
         Interp_LMC_Opti_Static102.txt
